## Mount Your Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Setup

*   List item
*   List item



In [2]:
!pip install -qU transformers==4.48.3 datasets==2.18.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb
!pip install -qU json-repair==0.29.1
# !pip install -qU faker==35.2.0
# !pip install -qU vllm==0.7.2

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

In [ ]:
from google.colab import userdata
import wandb

wandb.login(key=userdata.get('wandb'))
hf_token = userdata.get('huggingface')
!huggingface-cli login --token {hf_token}

## Imports

In [5]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

data_dir = "/gdrive/MyDrive/llm-finetuning"
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

device = "cuda"
torch_dtype = None #bf16

def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

## Tasks

In [ ]:
# story = """
# ذكرت مجلة فوربس أن العائلة تلعب دورا محوريا في تشكيل علاقة الأفراد بالمال،
#  حيث تتأثر هذه العلاقة بأنماط السلوك المالي المتوارثة عبر الأجيال.

# التقرير الذي يستند إلى أبحاث الأستاذ الجامعي شاين إنيت حول
# الرفاه المالي يوضح أن لكل شخص "شخصية مالية" تتحدد وفقا لطريقة
#  تفاعله مع المال، والتي تتأثر بشكل مباشر بتربية الأسرة وتجارب الطفولة.

#  الأبعاد الثلاثة للعلاقة بالمال
# بحسب الدراسة، هناك ثلاثة أبعاد رئيسية تشكّل علاقتنا بالمال:

# الاكتساب (A): يميل الأفراد الذين ينتمون لهذا
#  البعد إلى اعتبار المال سلعة قابلة للجمع، حيث يرون
# في تحقيق الثروة هدفا بحد ذاته. والجانب السلبي لهذا
#  النمط هو إمكانية التحول إلى هوس بالثروة أو العكس،
#  أي رفض تام لاكتساب المال باعتباره مصدرا للفساد.

# الاستخدام (U): يرى هؤلاء الأشخاص المال أداة للتمتع بالحياة، حيث يربطون قيمته بقدرته على توفير
# المتعة والراحة. ومع ذلك، قد يصبح
# البعض مدمنا على الإنفاق، في حين يتجه آخرون إلى التقشف المفرط خوفا من المستقبل.

# الإدارة (M): أصحاب هذا النمط يعتبرون المال مسؤولية تتطلب التخطيط الدقيق. لكن في بعض الحالات،
#  قد يتحول الأمر إلى هوس مفرط بإدارة الإنفاق، مما يؤثر سلبا على العلاقات الشخصية.

#  كيف تؤثر العائلة على علاقتنا بالمال؟
# يشير التقرير إلى أن التجارب الأسرية تلعب دورا رئيسيا في تحديد
#  "الشخصية المالية" لكل فرد، على سبيل المثال، إذا كان أحد الوالدين يعتمد على المال
# كمكافأة للسلوك الجيد، فقد يتبنى الطفل لاحقا النمط نفسه في حياته البالغة.

# لتحليل هذه التأثيرات بشكل دقيق، طورت رابطة العلاج المالي
# (Financial Therapy Association) أداة تسمى مخطط الجينوم المالي (Money Genogram)،
# وهو نموذج يُستخدم لتحديد الأنماط المالية داخل العائلة.

# تتضمن هذه الأداة:

# رسم شجرة عائلية.
# تصنيف أفراد العائلة وفقا للأبعاد الثلاثة للعلاقة بالمال (A ،U ،M).
# تحديد ما إذا كان السلوك المالي لكل فرد صحيا (+) أو غير صحي (-).
# على سبيل المثال، إذا نشأ شخص في عائلة
# اعتادت على الإنفاق المفرط، فقد يكون لديه ميل قوي إلى اتباع النمط نفسه،
#  أو العكس تماما، حيث يصبح مقتصدا بشكل مبالغ فيه كرد فعل نفسي.
# """

In [ ]:
# # # Another Sample

# story = """
# قرر المجلس القومي للأجور في مصر، زيادة الحد الأدنى لأجر العاملين بالقطاع الخاص إلى 7 آلاف جنيه شهريًا مقابل 6 آلاف جنيه، على أن يتم تطبيق الزيادة اعتبارًا من 1 مارس 2025.
# كما قرر المجلس أن يكون الحد الأدنى لقيمة العلاوة الدورية للعاملين بالقطاع الخاص 250 جنيهًا شهريًا، ولأول مرة يقرر المجلس القومي للأجور وضع حد أدنى للأجر للعمل المؤقت "جزء من الوقت"، بحيث لا يقل أجرهم عن 28 جنيهًا صافيًا في الساعة، وذلك وفقًا لتعريفهم الوارد في قانون العمل.
# وقالت وزيرة التخطيط والتنمية الاقتصادية والتعاون الدولي، رانيا المشاط، إن رفع الحد الأدنى للأجور يأتي في إطار الحرص على الاستجابة للمستجدات الاقتصادية الراهنة، بما يعزز الاستقرار الاقتصادي والاجتماعي، مضيفة أن ذلك يتسق مع المعايير الدولية، حيث تؤكد منظمة العمل الدولية على ضرورة مراجعة الحد الأدنى للأجور على أساس دوري، لحماية القوة الشرائية للأسر، واستيعاب التغيرات الاقتصادية التدريجية.
# """

In [6]:
story = """
أنهى مسئولو النادى الأهلى كل تفاصيل عقد أحمد مصطفى زيزو، جناح الزمالك، بحيث ينضم اللاعب لمدة 4 سنوات قادمة بدءا من الموسم الجديد، فى صفقة انتقال حر بعد نهاية عقده مع الزمالك عقب نهاية الموسم الحالى.

ووضع الأهلى زيزو فى الفئة الأولى بحيث يحصل على 25 مليون جنيه سنويا بالإضافة إلى عقد إعلانى مقابل 60 مليون جنيه، بخلاف 80 مليون جنيه أخرى سيحصل عليها دفعة واحدة من خارج النادى عند الإعلان الرسمى عن الصفقة.

وينتهى عقد أحمد سيد زيزو مع الزمالك بنهاية الموسم الجارى، مما يجعل له الحق فى التفاوض مع أى نادٍ وإتمام الاتفاق معه والانضمام له عقب نهاية عقده الحالي خلال يونيو المقبل.

وحصل زيزو على تأشيرة السفر لأمريكا بالفعل للمشاركة مع الأهلى في مونديال الأندية الصيف المقبل.

ويستعد الأهلى للمشاركة فى بطولة كأس العالم للأندية بشكلها الجديد فى الولايات المتحدة الأمريكية، والمقامة بمشاركة 32 ناديًا من أكبر فرق العالم
"""

### Details Extraction

In [ ]:
##  What I want from LLM

# {
#     "story_title": "",
#     "story_keywards": ["kw1", "kw2"]
#     "story_summary": "",
#     "story_entities": []
# }


In [7]:
StoryCategory = Literal[
    "politics", "sports", "art", "technology", "economy",
    "health", "entertainment", "science",
    "not_specified"
]


EntityType = Literal[
    "person-male", "person-female", "location", "organization", "event", "time",
    "quantity", "money", "product", "law", "disease", "artifact", "not_specified"
]

class Entity(BaseModel):
    entity_value: str = Field(..., description="The actual name or value of the entity.")
    entity_type: EntityType = Field(..., description="The type of recognized entity.")

class NewsDetails(BaseModel):
    story_title: str = Field(..., min_length=5, max_length=300,
                             description="A fully informative and SEO optimized title of the story.")

    story_keywords: List[str] = Field(..., min_items=1,
                                      description="Relevant keywords associated with the story.")

    story_summary: List[str] = Field(
                                    ..., min_items=1, max_items=5,
                                    description="Summarized key points about the story (1-5 points)."
                                )

    story_category: StoryCategory = Field(..., description="Category of the news story.")

    story_entities: List[Entity] = Field(..., min_items=1, max_items=10,
                                        description="List of identified entities in the story.")


In [8]:
details_extraction_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are an NLP data paraser.",
            "You will be provided by an Arabic text associated with a Pydantic scheme.",
            "Generate the ouptut in the same story language.",
            "You have to extract JSON details from text according the Pydantic details.",
            "Extract details as mentioned in text.",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps(
                NewsDetails.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Story Details:",
            "```json"
        ])
    }
]

In [ ]:
json.dumps(NewsDetails.model_json_schema(), ensure_ascii=False)

### Translation

In [ ]:
##  What I want from LLM

# {
#     "translated_title": "",
#     "translated_content": ""
# }

In [10]:
class TranslatedStory(BaseModel):
    translated_title: str = Field(..., min_length=5, max_length=300,
                                  description="Suggested translated title of the news story.")
    translated_content: str = Field(..., min_length=5,
                                    description="Translated content of the news story.")

targeted_lang = "English"

translation_messages = [
    {
        "role": "system",
        "content": "\n".join([
            "You are a professional translator.",
            "You will be provided by an Arabic text.",
            "You have to translate the text into the `Targeted Language`.",
            "Follow the provided Scheme to generate a JSON",
            "Do not generate any introduction or conclusion."
        ])
    },
    {
        "role": "user",
        "content":  "\n".join([
            "## Story:",
            story.strip(),
            "",

            "## Pydantic Details:",
            json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
            "",

            "## Targeted Language:",
            targeted_lang,
            "",

            "## Translated Story:",
            "```json"

        ])
    }
]

## Evaluation(base_model)

In [ ]:
model_temp = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer_temp = AutoTokenizer.from_pretrained(base_model_id)

In [12]:
model_temp

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [13]:
text = tokenizer_temp.apply_chat_template(
    details_extraction_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer_temp([text], return_tensors="pt").to(device)

generated_ids = model_temp.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer_temp.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [14]:
print(response)

{
    "story_title": "Ahmed Moustafa Zizou's Contract Ends",
    "story_keywords": [
        "Ahmed Moustafa Zizou",
        "contract ends",
        "Ahmed Sadek Zizou",
        "transfer fee",
        "Cup World Cup"
    ],
    "story_summary": [
        "Ahmed Moustafa Zizou, a player for Al Ahli, has completed all details related to his contract.",
        "Zizou is set to join Al Ahli for four years starting from the new season, after ending his contract with Al Ahli at the end of this season."
    ],
    "story_category": "sports",
    "story_entities": [
        {
            "entity_value": "Ahmed Moustafa Zizou",
            "entity_type": "person-male"
        },
        {
            "entity_value": "Al Ahli",
            "entity_type": "organization"
        }
    ]
}


In [15]:
text = tokenizer_temp.apply_chat_template(
    translation_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer_temp([text], return_tensors="pt").to(device)

generated_ids = model_temp.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer_temp.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [16]:
print(response)

{
  "translated_title": "Ahmed Moustafa Zizou's Contract Ends with Cairo Eagles",
  "translated_content": "The management of Al Ahli has finalized all details regarding Ahmed Moustafa Zizou's contract, which he will join for four years starting from next season. The deal is free transfer after his contract expires with Al Ahli at the end of this season."
}


## Evaluate OpenAI

In [ ]:
from openai import OpenAI
from google.colab import userdata

openai_client = OpenAI(
    api_key=userdata.get('openai-colab'),
    # base_url="http://localhost:8000"
)

openai_model_id = "gpt-4o-mini"

In [ ]:
chat_completion = openai_client.chat.completions.create(
    messages=details_extraction_messages,
    model=openai_model_id,
    temperature=0.2,
)

print(chat_completion.choices[0].message.content)

In [ ]:
chat_completion = openai_client.chat.completions.create(
    messages=translation_messages,
    model=openai_model_id,
    temperature=0.2,
)

print(chat_completion.choices[0].message.content)

In [ ]:
json.loads(chat_completion.choices[0].message.content) #type(chat_completion.choices[0].message.content)==>Str

## Knowledge Distillation

In [ ]:
raw_data_path = join(data_dir, "datasets", "news-sample.jsonl")

raw_data = []
for line in open(raw_data_path):
    if line.strip() == "":
        continue

    raw_data.append(
        json.loads(line.strip())
    )

random.Random(101).shuffle(raw_data)

print(f"Raw data: {len(raw_data)}")

Raw data: 2400


In [ ]:
raw_data[0]#['content']

In [ ]:
cloud_model_id = "gpt-4o-mini"
price_per_1m_input_tokens = 0.150
price_per_1m_output_tokens = 0.600

prompt_tokens = 0
completion_tokens = 0

save_to = join(data_dir, "datasets", "sft.jsonl")

ix = 0
for story in tqdm(raw_data):

    sample_details_extraction_messages = [
        {
            "role": "system",
            "content": "\n".join([
                "You are an NLP data paraser.",
                "You will be provided by an Arabic text associated with a Pydantic scheme.",
                "Generate the ouptut in the same story language.",
                "You have to extract JSON details from text according the Pydantic details.",
                "Extract details as mentioned in text.",
                "Do not generate any introduction or conclusion."
            ])
        },
        {
            "role": "user",
            "content": "\n".join([
                "## Story:",
                story['content'].strip(),
                "",

                "## Pydantic Details:",
                json.dumps(
                    NewsDetails.model_json_schema(), ensure_ascii=False
                ),
                "",

                "## Story Details:",
                "```json"
            ])
        }
    ]

    response = openai_client.chat.completions.create(
                            messages=sample_details_extraction_messages,
                            model=cloud_model_id,
                            temperature=0.2,
                        )

    if response.choices[0].finish_reason != "stop":
        prompt_tokens += response.usage.prompt_tokens
        continue

    llm_response = response.choices[0].message.content
    llm_resp_dict = parse_json(llm_response)

    if not llm_resp_dict:
        continue

    with open(save_to, "a", encoding="utf8") as dest:
        dest.write(json.dumps({
            "id": ix,
            "story": story['content'].strip(),
            "task": "Extrat the story details into a JSON.",
            "output_scheme": json.dumps( NewsDetails.model_json_schema(), ensure_ascii=False ),
            "response": llm_resp_dict,
        }, ensure_ascii=False, default=str)  + "\n" )

    ix += 1
    prompt_tokens += response.usage.prompt_tokens
    completion_tokens += response.usage.completion_tokens

    if(ix % 3) == 0:
        cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
        cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
        total_cost = cost_input + cost_output

        print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")

In [ ]:
cloud_model_id = "gpt-4o-mini"
price_per_1m_input_tokens = 0.150
price_per_1m_output_tokens = 0.600

prompt_tokens = 0
completion_tokens = 0

save_to = join(data_dir, "datasets", "sft.jsonl")

ix = 0
for story in tqdm(raw_data):

    for targeted_lang in ["English", "French"]:
        sample_translation_messages = [
            {
                "role": "system",
                "content": "\n".join([
                    "You are a professional translator.",
                    "You will be provided by an Arabic text.",
                    "You have to translate the text into the `Targeted Language`.",
                    "Follow the provided Scheme to generate a JSON",
                    "Do not generate any introduction or conclusion."
                ])
            },
            {
                "role": "user",
                "content": "\n".join([
                    "## Pydantic Details:",
                    json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
                    "",

                    "## Targeted Language or Dialect:",
                    targeted_lang,
                    "",

                    "## Story:",
                    story['content'].strip(),
                    "",

                    "## Translated Story:",
                    "```json"
                ])
            }
        ]

        response = openai_client.chat.completions.create(
                                messages=sample_translation_messages,
                                model=cloud_model_id,
                                temperature=0.2,
                            )

        if response.choices[0].finish_reason != "stop":
            prompt_tokens += response.usage.prompt_tokens
            continue

        llm_response = response.choices[0].message.content
        llm_resp_dict = parse_json(llm_response)

        if not llm_resp_dict:
            continue

        with open(save_to, "a", encoding="utf8") as dest:
            dest.write(json.dumps({
                "id": ix,
                "story": story['content'].strip(),

                "output_scheme": json.dumps( TranslatedStory.model_json_schema(), ensure_ascii=False ),
                "task": f"You have to translate the story content into {targeted_lang} associated with a title into a JSON.",

                "response": llm_resp_dict,
            }, ensure_ascii=False, default=str)  + "\n" )

        ix += 1
        prompt_tokens += response.usage.prompt_tokens
        completion_tokens += response.usage.completion_tokens

        if(ix % 50) == 0:
            cost_input = (prompt_tokens / 1_000_000) * price_per_1m_input_tokens
            cost_output = (completion_tokens / 1_000_000) * price_per_1m_output_tokens
            total_cost = cost_input + cost_output

            print(f"Iteration {ix}: Total Cost = ${total_cost:.4f} ")

## Format Finetuning Datasets

In [ ]:

sft_data_path = join(data_dir, "datasets", "sft.jsonl")
llm_finetunning_data = []

system_message = "\n".join([
    "You are a professional NLP data parser.",
    "Follow the provided `Task` by the user and the `Output Scheme` to generate the `Output JSON`.",
    "Do not generate any introduction or conclusion."
])

for line in open(sft_data_path):
    if line.strip() == "":
        continue

    rec = json.loads(line.strip())

    llm_finetunning_data.append({
        "system": system_message,
        "instruction": "\n".join([
            "# Story:",
            rec["story"],

            "# Task:",
            rec["task"],

            "# Output Scheme:",
            rec["output_scheme"],
            "",

            "# Output JSON:",
            "```json"

        ]),
        "input": "",
        "output": "\n".join([
            "```json",
            json.dumps(rec["response"], ensure_ascii=False, default=str),
            "```"
        ]),
        "history": []
    })

random.Random(101).shuffle(llm_finetunning_data)

In [ ]:
len(llm_finetunning_data)

2766

In [ ]:
train_sample_sz = 2700

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

os.makedirs(join(data_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "train.json"), "w") as dest:
    json.dump(train_ds, dest, ensure_ascii=False, default=str)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False, default=str)

In [ ]:
join(data_dir, "datasets", "llamafactory-finetune-data", "val.json")

'/gdrive/MyDrive/youtube-resources/llm-finetuning/datasets/llamafactory-finetune-data/val.json'

## Finetuning

In [ ]:
# # Configure LLaMA-Factory for the new datasets

# # update /content/LLaMA-Factory/data/dataset_info.json and append
# ```
   "news_finetune_train": {
        "file_name": "/gdrive/MyDrive/llm-finetuning/datasets/llamafactory-finetune-data/train.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    },
    "news_finetune_val": {
        "file_name": "/gdrive/MyDrive/llm-finetuning/datasets/llamafactory-finetune-data/val.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }
# ```

# https://wandb.ai/abdallahsamir735/llamafactory/runs/apwbkni9
# https://wandb.ai/abdallahsamir735/llamafactory/runs/c5tf0q90

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 64
lora_target: all

### dataset
dataset: news_finetune_train
eval_dataset: news_finetune_val
template: qwen
cutoff_len: 3500
# max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /gdrive/MyDrive/llm-finetuning/models/checkpoint-1500
output_dir: /gdrive/MyDrive/llm-finetuning/models/
logging_steps: 10
save_steps: 500
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100

report_to: wandb
run_name: newsx-finetune-llamafactory

push_to_hub: true
export_hub_model_id: "aaaaaaaaaaaaaaaaaa50D/news-analyzer"
hub_private_repo: true
hub_strategy: checkpoint


Overwriting /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/news_finetune.yaml

## New Finetuned Model Evaluation

In [17]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype = torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [18]:
finetuned_model_id = "/gdrive/MyDrive/llm-finetuning/models"
model.load_adapter(finetuned_model_id)

In [19]:
def generate_resp(messages):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

response = generate_resp(details_extraction_messages)
response2 = generate_resp(translation_messages)

In [20]:
parse_json(response)

{'story_title': 'أحمد مصطفى زيزو ينضم للاهلى لمدة 4 سنوات بعد عودة الزمالك',
 'story_keywords': ['أحمد مصطفى زيزو',
  'الاهلى',
  'الزمالك',
  'صفقة انتقال حر',
  'مونديال الأندية'],
 'story_summary': ['أحمد مصطفى زيزو ينضم للاهلى لمدة 4 سنوات.',
  'الصفقة تتضمن 25 مليون جنيه سنوية و60 مليون جنيه إعلانية.',
  'يزinho يحصل على 80 مليون جنيه أخرى بعد الإعلان الرسمي.',
  'يزinho حصل على تأشيرة السفر لأمريكا للمشاركة في كأس العالم.'],
 'story_category': 'sports',
 'story_entities': [{'entity_value': 'أحمد مصطفى زيزو',
   'entity_type': 'person-male'},
  {'entity_value': 'الاهلى', 'entity_type': 'organization'},
  {'entity_value': 'الزمالك', 'entity_type': 'organization'},
  {'entity_value': 'مونديال الأندية', 'entity_type': 'event'},
  {'entity_value': 'كأس العالم للأندية', 'entity_type': 'event'},
  {'entity_value': 'أمريكا', 'entity_type': 'location'}]}

In [21]:
parse_json(response2)

{'translated_title': "Ahmed Samy Zizou's Contract with Al Ahli Ends After Four Years",
 'translated_content': "The officials of Al Ahli have finalized all details regarding Ahmed Samy Zizou's contract with Zamalek, who is a midfielder for the Cairo club, allowing him to join the team for four years starting from the new season. The deal was made after the expiration of his contract with Zamalek at the end of this season.\n\nAl Ahli placed Zizou in the first category, which entitles him to earn 25 million pounds annually in addition to a sponsorship contract worth 60 million pounds, along with another 80 million pounds that he will receive immediately upon the official announcement of the deal outside the club.\n\nAhmed Seddiq Zizou's contract with Zamalek ends at the end of this season, giving him the right to negotiate with any club and finalize the agreement before the end of his current contract next June.\n\nZizou has already obtained a travel visa to America to participate with Al